In [2]:
import numpy as np 
import pandas as pd
from sklearn.metrics import accuracy_score , make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import preprocessing 
from sklearn.linear_model import LogisticRegression
import warnings 
warnings.filterwarnings("ignore")


In [3]:
path = "C:/Users/PC/Downloads/"

## Carga 

In [4]:
dataset = pd.read_csv(path + "iris_dataset.csv",header = 0,sep=";",decimal=",") 
prueba = pd.read_csv(path + "iris_prueba.csv",header = 0,sep=";",decimal=",")

In [5]:
dataset

,X1,X2,X3,X4,y
0,6.7,3.1,4.4,1.4,1
1,6.1,2.8,4.0,1.3,1
2,5.4,3.9,1.3,0.4,0
3,5.0,3.3,1.4,0.2,0
4,4.8,3.1,1.6,0.2,0
...,...,...,...,...,...
115,4.9,2.4,3.3,1.0,1
116,4.4,3.2,1.3,0.2,0
117,4.4,3.0,1.3,0.2,0
118,7.1,3.0,5.9,2.1,2


In [6]:
covariables = [x for x in dataset.columns if x not in ["y"] ]
X= dataset.get(covariables)
y = dataset["y"]

X_nuevo = prueba.get(covariables)
y_nuevo = prueba["y"]

## Entrenar el modelo

### Predecir Test

In [7]:
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size = 0.5)
Z_1=preprocessing.StandardScaler()
Z_1.fit(X_train)
X_train_Z= Z_1.transform(X_train)
X_test_Z= Z_1.transform(X_test)

modelo = LogisticRegression(random_state=123)#C parametro de regularizacion el modelo y se optiene el beta con ese parametro, valor optimo de c para tener el mejor acuracy
parametros ={'C': np.arange(0.1,5.1,0.1)}
grilla1 = GridSearchCV(estimator=modelo,param_grid=parametros,cv=5,scoring=make_scorer(accuracy_score),n_jobs=-1)
grilla1.fit(X_train_Z, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=123), n_jobs=-1,
             param_grid={'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. ])},
             scoring=make_scorer(accuracy_score))

In [8]:
grilla1.best_params_

{'C': 1.8000000000000003}

In [9]:
y_hat_test= grilla1.predict(X_test_Z)

### Predecir train

In [10]:
Z_2= preprocessing.StandardScaler()
Z_2.fit(X_test)
X_train_Z= Z_2.transform(X_train)
X_test_Z= Z_2.transform(X_test)

modelo2 = LogisticRegression(random_state=123)#C parametro de regularizacion el modelo y se optiene el beta con ese parametro, valor optimo de c para tener el mejor acuracy
parametros ={'C': np.arange(0.1,5.1,0.1)}
grilla2 = GridSearchCV(estimator=modelo2,param_grid=parametros,cv=5,scoring=make_scorer(accuracy_score),n_jobs=-1)
grilla2.fit(X_test_Z, y_test)

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=123), n_jobs=-1,
             param_grid={'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. ])},
             scoring=make_scorer(accuracy_score))

In [11]:
grilla2.best_params_

{'C': 3.8000000000000003}

In [12]:
y_hat_train= grilla2.predict(X_train_Z)

In [13]:
u1 = accuracy_score(y_test, y_hat_test)
u2 = accuracy_score(y_train, y_hat_train)
np.abs(u1-u2)< 10

True

In [14]:
u1 

0.9666666666666667

In [15]:
u2

0.9

In [16]:
if np.abs(u1-u2)< 10: #modelo no esta sobrentrenado
    modelo_completo= LogisticRegression(random_state=123)
    grilla_completo = GridSearchCV(estimator=modelo_completo,param_grid=parametros,cv=5,scoring=make_scorer(accuracy_score),n_jobs=-1)
    Z= preprocessing.StandardScaler()
    Z.fit(X)
    X_z= Z.transform(X)
    grilla_completo.fit(X_z,y)
else: #sino cumple, no optimice 
    grilla_completo = LogisticRegression(random_state=123)
    Z= preprocessing.StandardScaler()
    Z.fit(X)
    X_z= Z.transform(X)
    grilla_completo.fit(X_z,y)

### Predicciones

In [17]:
X_nuevo_Z= Z.transform(X_nuevo)
y_hat_nuevo = grilla_completo.predict(X_nuevo_Z)

### Evaluación modelo

In [18]:
accuracy_score(y_nuevo,y_hat_nuevo )

1.0

## 2.Pipeline

In [19]:
def load_data():
    covariables = [x for x in dataset.columns if x not in ["y"] ]
    X= dataset.get(covariables)
    y = dataset["y"]

    X_nuevo = prueba.get(covariables)
    y_nuevo = prueba["y"]
    
    return X, y, X_nuevo, y_nuevo 


In [20]:
def preprocessing_Z(X):
    Z= preprocessing.StandardScaler()
    Z.fit(X)
    X_z= Z.transform(X)
    return Z, X_z

In [21]:
def training_model(X,y):
    X_train, X_test, y_train, y_test= train_test_split(X,y, test_size = 0.5)
    #MODELO 1
    Z_1,X_train_Z= preprocessing_Z(X_train)
    X_test_Z= Z_1.transform(X_test)

    modelo1 = LogisticRegression(random_state=123)#C parametro de regularizacion el modelo y se optiene el beta con ese parametro, valor optimo de c para tener el mejor acuracy
    parametros ={'C': np.arange(0.1,5.1,0.1)}
    grilla1 = GridSearchCV(estimator=modelo1,param_grid=parametros,cv=5,scoring=make_scorer(accuracy_score),n_jobs=-1)
    grilla1.fit(X_train_Z, y_train)
    y_hat_test= grilla1.predict(X_test_Z)
    
    #MODELO 2
    Z_2,X_test_Z= preprocessing_Z(X_test)
    X_train_Z= Z_2.transform(X_train)

    modelo2 = LogisticRegression(random_state=123)#C parametro de regularizacion el modelo y se optiene el beta con ese parametro, valor optimo de c para tener el mejor acuracy
    grilla2 = GridSearchCV(estimator=modelo2,param_grid=parametros,cv=5,scoring=make_scorer(accuracy_score),n_jobs=-1)
    grilla2.fit(X_test_Z, y_test)
    y_hat_train= grilla2.predict(X_train_Z)
    
    u1 = accuracy_score(y_test, y_hat_test)
    u2 = accuracy_score(y_train, y_hat_train)
    
    if np.abs(u1-u2)< 10: #modelo no esta sobrentrenado
        modelo_completo= LogisticRegression(random_state=123)
        grilla_completo = GridSearchCV(estimator=modelo_completo,param_grid=parametros,cv=5,scoring=make_scorer(accuracy_score),n_jobs=-1)
        Z= preprocessing.StandardScaler()
        Z.fit(X)
        X_z= Z.transform(X)
        grilla_completo.fit(X_z,y)
    else: #sino cumple, no optimice 
        grilla_completo = LogisticRegression(random_state=123)
        Z= preprocessing.StandardScaler()
        Z.fit(X)
        X_z= Z.transform(X)
        grilla_completo.fit(X_z,y)
    #Predicciones   
    X_nuevo_Z= Z.transform(X_nuevo)
    y_hat_nuevo = grilla_completo.predict(X_nuevo_Z)
    #Evaluación modelo
    metrica= accuracy_score(y_nuevo,y_hat_nuevo)
    
    return metrica

In [22]:
def ML_system_regression():
    try: #
        X, y, X_nuevo, y_nuevo  = load_data()
        metric = training_model(X,y)
        return {'success':True, 'accuracy':metric }
    except Exception as e:
        return {'succes':False, 'message':str(e)}

In [26]:
resultado = ML_system_regression()

In [27]:
resultado["success"]

True

In [28]:
resultado["accuracy"]

1.0